In [1]:
import os
import collections
import pickle as cPickle
import collections
import numpy as np
from gensim import corpora, models, similarities
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics.pairwise import cosine_similarity
import operator
f = open(os.path.join('dataset', 'tips.data'), 'rb')
data = cPickle.load(f)


train_s=['Train_10','Train_20','Train_50','Train_100']
test_s=['Test_10','Test_20','Test_50','Test_100']
test_poiT_s=['True_10','True_20','True_50','True_100']

stoplist = set('for a an of the and to in is are it you I me'.split())

def evaluate_score():
    #evaluate the score for each user
    a=[]
    for i in data[user][test_poiT]:
        count=len(data[user][test])
        for poi,score_sim in sorted_POI_dict: 
            
            if poi ==i :
                if (count > (len(data[user][test])-len(data[user][test_poiT]))):
                    a.append(len(data[user][test]))
                    
                else:
                    a.append(count)
                                     
            count = count - 1
    return sum(a)/len(data[user][test_poiT])*(1.0/len(data[user][test]))

def Collection_words(_user,dd,_poi):
    texts=[[''.join(e for e in word if e.isalpha()) for word in document.lower().split() if word not in stoplist] 
            for document in data[_user][dd][_poi]]
    #remove ''space word
    texts = [[word for word in text if word not in ''] for text in texts]
    aList=[]
    for document in texts:
        for word in document:
            aList.append(word)
    return aList        

 

c:\users\alwa\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
output = {}
for train,test,test_poiT in zip(train_s,test_s,test_poiT_s):
    usersocre_dict = collections.defaultdict(list)
    for user in data.keys():
        user_texts=[]
        #######collect future word from each comment to construct dictionary and corpus
        for poi in data[user][train].keys():
            train_list= Collection_words(user,train,poi)
            user_texts.append(train_list)
        
        dictionary = corpora.Dictionary(user_texts)
        corpus = [dictionary.doc2bow(text) for text in user_texts]
        #######
        
        ###rank the test sample by the similarity with train sample
        POI_dict = collections.defaultdict(list)
        for poi in data[user][test].keys():
            test_list=Collection_words(user,test,poi)
            new_vec = dictionary.doc2bow(test_list)   
            lsi = models.LsiModel(corpus, id2word=dictionary)
            vec_lsi = lsi[new_vec] # convert the query to LSI space
            index = similarities.MatrixSimilarity(lsi[corpus])
            sims = index[vec_lsi] # perform a similarity query against the corpus
            POI_dict[poi]=sum(sims)
    
        sorted_POI_dict = sorted(POI_dict.items(), key=operator.itemgetter(1),reverse=True)
        ###
        

        usersocre_dict[user] = evaluate_score() 
#     break
    output[train]=usersocre_dict   

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)], [(14, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)], [(8, 1), (20, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)], [(0, 1), (9, 1), (14, 1), (27, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1)], [(61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1)], [(9, 1), (18, 1), (52, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 2)], [(50, 1), (55, 1), (93, 1), (94, 2), (95, 2), (96, 1), (97, 1), (98, 1), (

In [3]:
output


{'Train_10': defaultdict(list,
             {72: 0.5,
              102: 0.2,
              104: 0.4,
              334: 1.0,
              429: 0.7000000000000001,
              616: 0.6000000000000001,
              953: 0.1,
              1317: 1.0,
              1319: 0.6000000000000001,
              1342: 1.0,
              2448: 0.9,
              4787: 0.6000000000000001,
              5009: 0.2,
              5183: 0.6000000000000001,
              5503: 0.7000000000000001,
              7599: 0.6000000000000001,
              8051: 0.6000000000000001,
              8388: 0.4,
              8466: 0.2,
              9962: 0.30000000000000004,
              11075: 0.30000000000000004,
              11466: 0.9,
              13095: 0.5,
              13999: 1.0,
              15416: 0.1,
              16255: 0.1,
              16602: 0.30000000000000004,
              17435: 0.4,
              19335: 0.1,
              19732: 0.4,
              20537: 0.6000000000000001,
        

In [4]:
score={}
for train in train_s:
    score[train]=sum(output[train].values())/len(output[train])
    
    

In [5]:
score

{'Train_10': 0.5565789473684213,
 'Train_100': 0.4758157894736842,
 'Train_20': 0.48059210526315804,
 'Train_50': 0.47389473684210526}

### Word2Vec

In [3]:
output = {}
for train,test,test_poiT in zip(train_s,test_s,test_poiT_s):
    usersocre_dict = collections.defaultdict(list)
    for user in data.keys():
        user_texts=[]
        #######collect future word from each comment to construct dictionary and corpus
        for poi in data[user][train].keys():
            train_list= Collection_words(user,train,poi)
            user_texts.append(train_list)
        
        dictionary = corpora.Dictionary(user_texts)
        corpus = [dictionary.doc2bow(text) for text in user_texts]
        #######
#         break
        word2vec = models.Word2Vec(user_texts, size=100, window=5, min_count=1, workers=4)
        user_w2v = np.zeros(100)
        for sentence in user_texts:
            tmp = np.zeros(100)
            for word in sentence:
#                 print(word, sentence)
                tmp += np.array(word2vec[word])
            tmp /= len(sentence)
            user_w2v += tmp
        user_w2v /= len(user_texts)
#         print(user_w2v)
        ###rank the test sample by the similarity with train sample
        POI_dict = collections.defaultdict(list)
        for poi in data[user][test].keys():
            test_list=Collection_words(user,test,poi)
#             new_vec = dictionary.doc2bow(test_list)   
            poi_sentence = np.zeros(100)
            for word in test_list:
                if word in word2vec:
                    poi_sentence += np.array(word2vec[word])
#             print(poi_sentence)
            POI_dict[poi]= cosine_similarity(poi_sentence.reshape(1,-1), user_w2v.reshape(1,-1))
        
        sorted_POI_dict = sorted(POI_dict.items(), key=operator.itemgetter(1),reverse=True)
        ###
        
    
        usersocre_dict[user] = evaluate_score() 
    
    output[train]=usersocre_dict   

In [4]:
score={}
for train in train_s:
    score[train]=sum(output[train].values())/len(output[train])
score

{'Train_10': 0.5657894736842105,
 'Train_100': 0.5134473684210527,
 'Train_20': 0.4960526315789473,
 'Train_50': 0.505842105263158}